In [ ]:
!unzip -q "data.zip" -d "./"

In [ ]:
!gdown --id '1OlSAVpo1k7__G4ToTSVMWrnNZIIPRe1E'
# 列出目前目錄下所有的檔案
!ls


Downloading...
From: https://drive.google.com/uc?id=1OlSAVpo1k7__G4ToTSVMWrnNZIIPRe1E
To: /content/data.zip
239MB [00:01, 162MB/s]
data.zip  sample_data


In [ ]:
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [ ]:
import pandas as pd
from tqdm import tqdm
import gc
import numpy as np
import lightgbm as lgb
# from calc_uauc import uAUC
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 100)
tqdm.pandas(desc='pandas bar')


In [ ]:

def get_count(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'count')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def get_sum(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'sum')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df

In [ ]:
train_list = ['05-14.pkl', '05-15.pkl', '05-16.pkl', '05-17.pkl', '05-18.pkl']
train = pd.DataFrame()
path = './new_train/'
for i in os.listdir(path):
    if i in train_list:
        print(i)
        train_temp = pd.read_pickle(path + i)
        train = pd.concat([train, train_temp], axis=0)
# train = pd.read_csv('../data/train/05-17.csv', header=None)
# train.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
# 'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
# train = reduce_mem(train)
#
# train1 = pd.read_csv('../data/train/05-18.csv', header=None)
# train1.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
# 'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
# train1 = reduce_mem(train1)

test = pd.read_csv('./phm_test.csv', header=None)
test.columns = ['time','gun_no','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
test = reduce_mem(test)

503.54 Mb, 165.94 Mb (67.05 %)


In [ ]:
train.head(5)

""


In [ ]:
train.columns

Index([], dtype='object')

In [ ]:
train['W_Error'].value_counts()

In [ ]:
# data = pd.concat([train, train1], axis=0)
# 处理标签
train['W_Error'][train['W_Error']==0] = '0'
# print(train['W_Error'].value_counts())
# train['task1_label'] = 1
train['task1_label'] = train['W_Error'].apply(lambda x:0 if x=='0' else 1)
print(train['task1_label'].value_counts())

# train1['W_Error'][train1['W_Error']==0] = '0'
# train1['task1_label'] = train1['W_Error'].apply(lambda x:0 if x=='0' else 1)
# print(train1['task1_label'].value_counts())

train['task2_label'] = train['W_Error'].apply(lambda x:1 if x=='E016' else 0)
train['task2_label'].value_counts()

In [ ]:
drop_columns = ['time', 'gun_no','area']
train.drop(drop_columns, axis=1, inplace=True)
test.drop(drop_columns, axis=1, inplace=True)
# train1.drop(drop_columns, axis=1, inplace=True)
train.head(5)

,W_Error,C_Cylinder_force,C_Differential_pressure,W_Friction,W_Maximum_aperture,W_Maximum_electrode_force,W_Start_friction,W_us2,W_Welding_point_count,W_position_count,in_Counterbalance_pressure,in_Electrode_force,in_Electrode_position,in_Sheet_thickness,in_Velocity,in_force_build_up,out_Cap_offset,out_Electrode_force,out_Electrode_position,out_force_build_up,task1_label,task2_label
0,0,3142,-4.539062,-63,157.125,6598,202,1,349001,22679640,6.414062,3000,25.0,2.699219,1,1,0.000000,3055,1.299805,1,0,0
1,0,44,0.939941,-10,159.750,6074,265,1,349011,2074682,14.992188,3000,50.0,2.699219,1,0,0.000000,0,50.093750,0,0,0
2,0,-65,0.479980,22,307.000,4350,244,1,29057,20691084,15.992188,2200,220.0,1.500000,1,0,2.900391,0,219.375000,0,0,0
3,0,30,1.049805,-132,158.125,5147,130,1,2670150,15726143,14.992188,2000,50.0,1.400391,1,0,3.400391,0,49.906250,0,0,0
4,0,-13,0.459961,-1,245.000,5144,-66,1,134293,13174955,10.671875,3600,210.0,2.000000,1,1,1.099609,370,205.000000,0,0,0


In [ ]:
# w_error = train.pop('W_Error')
# task1_label = train.pop('task1_label')

In [ ]:
# train, y_train, X_test, y_test = train_test_split(train, train['task1_label'])
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
oof_lgb = np.zeros(len(train))
prediction = np.zeros(len(test))
feature_importance_df = pd.DataFrame()
col = [tmp_col for tmp_col in train.columns if tmp_col not in ['W_Error', 'task1_label', 'task2_label']]
X_train = train[col].values
# print(train[col].info())
y_train = train['task1_label'].values

In [ ]:
def f2_eval(preds, dtrain):
    labels = dtrain.get_label()
    y_pred = np.where(preds > 0.3, 1, 0)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    # print(precision, recall)
    f2_score = 5 * precision * recall / (4 * precision + recall)
    return 'score', f2_score, True

param = {'num_leaves': 48,
         # 'min_data_in_leaf': 30,
         'objective': 'binary',
         'learning_rate': 0.02,
         # 'max_depth': 9,
         # "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         # "bagging_freq": 1,
         "bagging_fraction": 0.9,
         # "bagging_seed": 11,
         "metric": 'auc',
         # "lambda_l1": 0.1,
         # "lambda_l2": 0.1,
         "verbosity": -1}

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 2000
    clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=200,
                    early_stopping_rounds=100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    prediction += clf.predict(test[col].values, num_iteration=clf.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = col
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')#返回各个特征的重要性
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
# oof_lgb2 += clf.predict(df_remove[col].values, num_iteration=clf.best_iteration)
# oof_lgb_final = np.argmax(oof_lgb, axis=1)
#     predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
print('feature importance:')
print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))

In [ ]:
label_lgb = np.where(oof_lgb > 0.3, 1, 0)

label_test = np.where(prediction > 0.3, 1, 0)

In [ ]:
def f2_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    # print(precision, recall)
    f2_score = 5 * precision * recall / (4 * precision + recall)
    return f2_score
# print('f1_score::{}'.format(f1_score(y_train, label_lgb)))
print(f2_score(y_train, label_lgb))
# print(f2_score(train1['task1_label'].values, label_test))
# print('test_auc:{}'.format(roc_auc_score(y_test, prediction)))
print(pd.DataFrame(label_lgb)[0].value_counts())

f1_score::0.9803695008721574
0.961583058375496 0.9999046317194901
0.9919978950602849
0    4855040
1     457949
Name: 0, dtype: int64


In [ ]:
prob = pd.DataFrame(label_test)
prob.columns = ['label']
prob.to_csv('../submission/predict_error_nodeal_task1.csv', index=None, header=None)
print(prob['label'].value_counts())

res_temp = pd.read_csv('../submission/res_temp.csv')
res = pd.concat([res_temp, prob], axis=1)
res.to_csv('../submission/task1.csv', index=None)

0    2798847
1     201153
Name: label, dtype: int64


In [ ]:
# train_opt = pd.DataFrame()
# for i in os.listdir(path):
#     if i not in train_list:
#         print(i)
#         train_temp = pd.read_pickle(path + i)
#         train_temp.drop(drop_columns, axis=1, inplace=True)
#         train_opt = pd.concat([train_opt, train_temp], axis=0)
#
# train_opt['W_Error'][train_opt['W_Error']==0] = '0'
# train_opt['task1_label'] = train_opt['W_Error'].apply(lambda x:0 if x=='0' else 1)
# # print(train_opt['task1_label'].value_counts())
# train_opt['task2_label'] = train_opt['W_Error'].apply(lambda x:1 if x=='E016' else 0)
# print(train_opt['task2_label'].value_counts())
# train_opt = train_opt[train_opt['task2_label']==1].reset_index()
# print(train_opt.shape)
# train = pd.concat([train, train_opt], axis=0).reset_index()
oof_lgb_task2 = np.zeros(len(train))
prediction_task2 = np.zeros(len(test))
print(train['task2_label'].value_counts())

04-25.pkl
04-26.pkl
04-27.pkl
04-28.pkl
04-29.pkl
04-30.pkl
05-01.pkl
05-02.pkl
05-03.pkl
05-04.pkl
05-05.pkl
05-06.pkl
05-07.pkl
05-08.pkl
05-09.pkl
05-10.pkl
05-11.pkl
05-12.pkl
05-13.pkl
0    17176567
1        1528
Name: task2_label, dtype: int64
(1528, 23)
0    5312263
1       3782
Name: task2_label, dtype: int64


#问题二

In [ ]:
y_train_task2 = train['task2_label'].values
X_train = train[col].values
# test.drop(drop_columns, axis=1, inplace=True)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train_task2)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train_task2[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train_task2[val_idx])

    num_round = 2000
    clf_task2 = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=200,
                    early_stopping_rounds=100)
    oof_lgb_task2[val_idx] = clf_task2.predict(X_train[val_idx], num_iteration=clf_task2.best_iteration)
    prediction_task2 += clf_task2.predict(test[col].values, num_iteration=clf_task2.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = col
    fold_importance_df["importance"] = clf_task2.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
# oof_lgb2 += clf.predict(df_remove[col].values, num_iteration=clf.best_iteration)
# oof_lgb_final = np.argmax(oof_lgb, axis=1)
#     predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
print('feature importance:')
print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))

fold n°1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[22]	training's auc: 0.998009	valid_1's auc: 0.99999
fold n°2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	training's auc: 1	valid_1's auc: 1
fold n°3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1
fold n°4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 0.999999
fold n°5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[41]	training's auc: 0.999998	valid_1's auc: 1
feature importance:
feature
C_Differential_pressure       1.925940e+07
out_Electrode_force           7.530250e+06
in_Counterbalance_pressure    4.531480e+06
C_Cylinder_force              3.080993e+06
in_Electrode_force            1.848391e+06
W_

In [ ]:
print(train['task2_label'].value_counts())
label_lgb_task2 = np.where(oof_lgb_task2 > 0.3, 1, 0)
label_test_task2 = np.where(prediction_task2 > 0.3, 1, 0)
print(f2_score(y_train_task2, label_lgb_task2))

0    5312263
1       3782
Name: task2_label, dtype: int64
0.9784974093264248 0.9986779481755684
0.9945755213819253


In [ ]:
prob_task2 = pd.DataFrame(label_test_task2)
prob_task2.columns = ['label']
prob_task2.to_csv('../submission/predict_e016_nodeal.csv', index=None, header=None)
print(prob_task2['label'].value_counts())

res_temp = pd.read_csv('../submission/res_temp.csv')
res = pd.concat([res_temp, prob_task2], axis=1)
res.to_csv('../submission/task2.csv', index=None)


0    2996089
1       3911
Name: label, dtype: int64


In [ ]:
res.head(5)


,time,gun_no,label
0,1621382400,5509057-3052,0
1,1621382400,5509057-3052,0
2,1621382401,5509057-3052,0
3,1621382401,5509057-3052,0
4,1621382402,5509057-3052,0


In [ ]:
res1 = res
# print(res['label'].value_counts())
error_res = res1[res1['label']==1]
print(len(error_res))
print(error_res['label'].index)
num = 0
for i in error_res['label'].index:
    num += 1
    if num%500==0:
        print(num)
    time = res1.iloc[i,0]
    gun_no = res1.iloc[i,1]
    res1['label'][(res1['time']>time-600) & (res1['time']<time) & (res1['gun_no']==gun_no)]=1


3572
Int64Index([   6296,    6323,    6331,    6347,    6361,    6371,    6382,
               6395,    6397,    6409,
            ...
            2813217, 2813231, 2813233, 2813250, 2813259, 2813274, 2813290,
            2813308, 2813320, 2813461],
           dtype='int64', length=3572)
500
1000
1500
2000
2500
3000
3500


In [ ]:
print(res1['label'].value_counts())
res1['label'].to_csv('../res/predict_e016.csv', index=None, header=None)

0    2816767
1     183233
Name: label, dtype: int64
